## YOLOv5 모델 (PyTorch)

* https://github.com/ultralytics/yolov5
* https://www.ultralytics.com

### 모델 다운로드

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

import torch
from IPython.display import Image, clear_output

/content
fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5


## 안전모 탐지 모델

### 데이터셋 다운로드

* 안전모 데이터셋: https://public.roboflow.com/object-detection/hard-hat-workers

In [ ]:
%mkdir /content/yolov5/hardhat
%cd /content/yolov5/hardhat
!curl -L "https://public.roboflow.com/ds/Wx8Z23YHmT?key=8hV1KTBqY4" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

mkdir: cannot create directory ‘/content/yolov5/hardhat’: File exists
/content/yolov5/hardhat
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   901  100   901    0     0   1206      0 --:--:-- --:--:-- --:--:--  1204
100  228M  100  228M    0     0  24.6M      0  0:00:09  0:00:09 --:--:-- 30.5M
Archive:  roboflow.zip
replace test/images/005553_jpg.rf.00c2cb19e836866919c22cdf205c6353.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
from glob import glob

train_img_list = glob('/content/yolov5/hardhat/train/images/*.jpg')
test_img_list = glob('/content/yolov5/hardhat/test/images/*.jpg')
print(len(train_img_list), len(test_img_list))

5269 1766


In [ ]:
from sklearn.model_selection import train_test_split

test_img_list, val_img_list = train_test_split(test_img_list, test_size=0.5, random_state=777)
print(len(test_img_list), len(val_img_list))

883 883


In [ ]:
import yaml

with open('/content/yolov5/hardhat/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('/content/yolov5/hardhat/test.txt', 'w') as f:
  f.write('\n'.join(test_img_list) + '\n')

with open('/content/yolov5/hardhat/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [ ]:
%cat /content/yolov5/hardhat/data.yaml

train: ../train/images
val: ../valid/images

nc: 3
names: ['head', 'helmet', 'person']

In [ ]:
%%writetemplate /content/yolov5/hardhat/data.yaml

train: ./hardhat/train/images
test: ./hardhat/test/images
val: ./hardhat/test/images

nc: 3
names: ['head', 'helmet', 'person']

UsageError: Cell magic `%%writetemplate` not found.


In [ ]:
%cat /content/yolov5/hardhat/data.yaml

### 모델 구성

In [ ]:
with open('data.yaml', 'r') as stream:
  num_classes = str(yaml.safe_load(stream)['nc'])

%cat /content/yolov5/models/yolov5s.yaml

In [ ]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# Parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
%cat /content/yolov5/models/custom_yolov5s.yaml

### 학습(Training)

* `img`: 입력 이미지 크기 정의
* `batch`: 배치 크기 결정
* `epochs`: 학습 기간 개수 정의
* `data`: yaml 파일 경로
* `cfg`: 모델 구성 지정
* `weights`: 가중치에 대한 경로 지정
* `name`: 결과 이름
* `nosave`: 최종 체크포인트만 저장
* `cache`: 빠른 학습을 위한 이미지 캐시

In [ ]:
%time
%cd /content/yolov5
!python train.py --img 416 --batch 64 --epochs 50 --data ./hardhat/data.yaml --cfg ./models/custom_yolov5s.yaml --weights '' --name hardhat_results --cache

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!ls runs/train/hardhat_results

In [ ]:
Image(filename='runs/train/hardhat_results/results.png', width=1000)

In [ ]:
Image(filename='runs/train/hardhat_results/train_batch0.jpg', width=1000)

In [ ]:
Image(filename='runs/train/hardhat_results/val_batch0_labels.jpg', width=1000)

### 검증(Validation)

In [ ]:
!python val.py --weights runs/train/hardhat_results/weights/best.pt --data ./hardhat/data.yaml --img 416 --iou 0.65 --half

In [ ]:
!python val.py --weights runs/train/hardhat_results/weights/best.pt --data ./hardhat/data.yaml --img 416 --task test

### 추론(Inference)

In [ ]:
%ls runs/train/hardhat_results/weights

In [ ]:
!python detect.py --weights runs/train/hardhat_results/weights/best.pt --img 416 --conf 0.4 --source ./hardhat/test/images

In [ ]:
import glob

image_name = random.choice(glob.glob('runs/detect/exp4/*.jpg'))
display(Image(filename=image_name))

### 모델 내보내기

In [ ]:
%mkdir /content/drive/My\ Drive/hardhat
%cp runs/train/hardhat_results/weights/best.pt /content/drive/My\ Drive/hardhat